In [1]:
import os
import glob

import plotly.express as px
import plotly.graph_objects as go

In [2]:
PRJ_DIR = os.path.join('/home', 'jovyan', 'work')
TESTSET_DIR = os.path.join(PRJ_DIR, '01_testset')

In [3]:
def read_shapes(seed):
    problem_path = os.path.join(TESTSET_DIR, 'in', '{:0>4}.txt'.format(seed)) 

    shapes = []

    with open(problem_path, 'r') as f:
        D = int(f.readline())  

        for i in range(2):
            planes = []

            for p in range(2):
                plane = []

                for z in range(D):
                    vals = list(map(int, f.readline().rstrip()))
                    plane.append(vals)

                planes.append(plane)

            shapes.append(planes)
    
    return D, shapes

In [4]:
def read_blocks(iter_count):
    blocks = []    
    
    if iter_count == -1:
        block_path = os.path.join(PRJ_DIR, '03_min_block', 'search_log', 'init.txt')
    else:
        block_path = os.path.join(PRJ_DIR, '03_min_block', 'search_log', 'iter_{}.txt'.format(iter_count))

    with open(block_path, 'r') as f:
        # 先頭はコメント
        comment = f.readline().rstrip()

        B = int(f.readline())

        for b in range(2):
            line = f.readline().rstrip().split()
            vals = list(map(int, line))

            block = []
            i = 0

            for x in range(D):
                plane = []

                for y in range(D):
                    z_vals = []

                    for z in range(D):
                        z_vals.append(vals[i])
                        i += 1

                    plane.append(z_vals)

                block.append(plane)

            blocks.append(block)    
    
    return comment, B, blocks

In [5]:
def show_shape(shape_index):
    planes = shapes[shape_index]

    fig = go.Figure()

    for z in range(D):
        # zx平面のシルエット
        for x in range(D):
            s = planes[0][z][x]

            x_list = [x, x + 1, x, x + 1, x, x + 1, x, x + 1]
            y_list = [-1, -1, -1, -1, -0.9, -0.9, -0.9, -0.9]
            z_list = [-z, -z, -z - 1, -z - 1, -z, -z, -z - 1, -z - 1]

            c = 'white' if s == 0 else 'gray'
            fig.add_trace(go.Mesh3d(x=x_list, y=y_list, z=z_list, alphahull=0, color=c, opacity=0.9))

        # zy平面のシルエット
        for y in range(D):
            s = planes[1][z][y]

            x_list = [-1, -1, -1, -1, -0.9, -0.9, -0.9, -0.9]
            y_list = [y, y + 1, y, y + 1, y, y + 1, y, y + 1]
            z_list = [-z, -z, -z - 1, -z - 1, -z, -z, -z - 1, -z - 1]

            c = 'white' if s == 0 else 'gray'
            fig.add_trace(go.Mesh3d(x=x_list, y=y_list, z=z_list, alphahull=0, color=c, opacity=0.9))

    # Block        
    block = blocks[shape_index]

    color_map = px.colors.cyclical.HSV
    color_size = len(color_map)

    for x in range(D):
        for y in range(D):
            for z in range(D):
                cell = block[x][y][z]

                if cell == 0:
                    continue

                x_list = [x ,  x+1, x  , x    , x+1, x+1 , x   , x+1]
                y_list = [y ,  y  , y+1, y    , y+1, y   , y+1 , y+1]
                z_list = [-z, -z  , -z , -z-1, -z , -z-1, -z-1, -z-1]

                if B >= color_size:
                    color = color_map[(cell-1) % color_size]
                else:
                    color = color_map[(cell-1) * color_size // B]

                fig.add_trace(go.Mesh3d(x=x_list, y=y_list, z=z_list, alphahull=0, color=color, opacity=0.8))



    fig.update_layout(
        scene = dict(
            xaxis=dict(range=[-1, D+1]),
            yaxis=dict(range=[-1, D+1]),
            #zaxis=dict(range=(0, D+1)),    
        )
    )

    fig.show()

In [6]:
seed = 8
D, shapes = read_shapes(seed)

search_log_list = []

# 初期状態
comment, B, blocks = read_blocks(-1)
#search_log_list.append((comment, B, blocks))

log_count = len(glob.glob(os.path.join(PRJ_DIR, '03_min_block', 'search_log', 'iter_*txt')))

for i in range(log_count):
    comment, B, blocks = read_blocks(i)
    search_log_list.append((comment, B, blocks))

In [7]:
for i in range(log_count):
    comment,_,_ = search_log_list[i]
    if 'Updated=1' in comment:
        print(comment)

# Iter=0,Trans=6,Score=12,NextScore=11,Updated=1
# Iter=1,Trans=6,Score=11,NextScore=10,Updated=1
# Iter=2,Trans=6,Score=10,NextScore=9,Updated=1
# Iter=3,Trans=6,Score=9,NextScore=8,Updated=1
# Iter=4,Trans=0,Score=8,NextScore=4,Updated=1
# Iter=5,Trans=6,Score=4,NextScore=2,Updated=1
# Iter=6,Trans=6,Score=2,NextScore=1,Updated=1
# Iter=196,Trans=0,Score=1,NextScore=1,Updated=1
# Iter=420,Trans=0,Score=1,NextScore=1,Updated=1
# Iter=428,Trans=0,Score=1,NextScore=1,Updated=1
# Iter=712,Trans=0,Score=1,NextScore=1,Updated=1
# Iter=836,Trans=0,Score=1,NextScore=1,Updated=1
# Iter=932,Trans=0,Score=1,NextScore=1,Updated=1


In [8]:
iter_index = 196
comment, B, blocks = search_log_list[iter_index]

In [9]:
# ブロック数のチェック
for b in range(1, B+1):
    cnt_1 = 0
    cnt_2 = 0
    
    block_1 = blocks[0]
    block_2 = blocks[1]
    
    for x in range(D):
        for y in range(D):
            for z in range(D):
                cell_1 = block_1[x][y][z]
                
                if cell_1 == b:
                    cnt_1 += 1
                    
                cell_2 = block_2[x][y][z]
                
                if cell_2 == b:
                    cnt_2 += 1
        
    if cnt_1 != cnt_2:
        print(b, cnt_1, cnt_2)

In [10]:
comment

'# Iter=196,Trans=0,Score=1,NextScore=1,Updated=1'